In [ ]:
# Info on how to get your api key (kaggle.json) here: https://github.com/Kaggle/kaggle-api#api-credentials
# Valeria Quiroz  - Gabriel Zapata
%%time
!pip install kaggle
api_token = {"username":"gabrielzapata","key":"45d11139cb4e32995898eae373213b5f"}
import json
import zipfile
import os
with open('/content/kaggle.json', 'w') as file:
    json.dump(api_token, file)
!mkdir /root/.kaggle      
!cp /content/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v/content
!chmod 600 /root/.kaggle/kaggle.json
!kaggle competitions download -c ga-customer-revenue-prediction
if not os.path.exists("/content/competitions/ga-customer-revenue-prediction"):
    os.makedirs("/content/competitions/ga-customer-revenue-prediction")
os.chdir('/content/competitions/ga-customer-revenue-prediction')
for file in os.listdir():
    zip_ref = zipfile.ZipFile(file, 'r')
    zip_ref.extractall()
    zip_ref.close()

- path is now set to: /content
 96% 321M/333M [00:04<00:00, 51.1MB/s]
100% 333M/333M [00:04<00:00, 76.4MB/s]
  0% 0.00/5.23M [00:00<?, ?B/s]
100% 5.23M/5.23M [00:00<00:00, 86.0MB/s]
  0% 0.00/2.55M [00:00<?, ?B/s]
100% 2.55M/2.55M [00:00<00:00, 173MB/s]
 86% 49.0M/56.8M [00:00<00:00, 70.3MB/s]
100% 56.8M/56.8M [00:00<00:00, 83.1MB/s]
 99% 1.11G/1.11G [00:15<00:00, 104MB/s]
100% 1.11G/1.11G [00:15<00:00, 77.1MB/s]
 67% 41.0M/61.1M [00:00<00:00, 44.5MB/s]
100% 61.1M/61.1M [00:00<00:00, 83.7MB/s]
CPU times: user 1min 37s, sys: 34.3 s, total: 2min 11s
Wall time: 5min 18s


### **Librerias**

In [ ]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from pandas import json_normalize
import seaborn as sns 
from sklearn import model_selection, preprocessing, metrics
import lightgbm as lgb
import plotly.graph_objs as go
from plotly import subplots
import plotly.offline as py
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
import datetime
from sklearn.model_selection import  GroupKFold
from sklearn.pipeline import Pipeline
#from sklearn.metrics import mean_absolute_percentage_error

### **Explicacion del codigo json sacado de Kaggle, para aplanar los datos**

In [ ]:
def load_df(csv_path='/content/competitions/ga-customer-revenue-prediction/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

In [ ]:
%%time
train_df = load_df()
#test_df = load_df('/content/competitions/ga-customer-revenue-prediction/test.csv')

Loaded train.csv. Shape: (903653, 55)
CPU times: user 1min 57s, sys: 7.79 s, total: 2min 5s
Wall time: 2min 5s


In [ ]:
train_df = train_df.drop( ['sessionId'] + ["trafficSource.campaignCode"], axis=1)
print(train_df.shape)

### **Eliminemos columnas constantes e imputar valores cero a NAN**

In [ ]:
print ("Antes de eliminar columnas constantes - shape de train y test datasets: ", train_df.shape)
train_df = train_df.loc[:, (train_df != train_df.iloc[0]).any()]
print ("Despues de eliminar columnas constantes - shape de train y test datasets: ", train_df.shape)

In [ ]:
# Replace NaN values throughout train dataset
train_df.replace(to_replace=np.nan, value=0, inplace=True)

### **Conversion variable categórica a númerica**

In [ ]:
# etiqueta codifica las variables categóricas y convierte las variables numéricas flotantes
cat_cols = ["channelGrouping", "device.browser",
            "device.deviceCategory", "device.operatingSystem", "device.isMobile",
            "geoNetwork.city", "geoNetwork.continent", 
            "geoNetwork.country", "geoNetwork.metro",
            "geoNetwork.networkDomain", "geoNetwork.region", 
            "geoNetwork.subContinent", "trafficSource.adContent", 
            "trafficSource.adwordsClickInfo.adNetworkType", 
            "trafficSource.adwordsClickInfo.gclId", 
            "trafficSource.adwordsClickInfo.page", 
            "trafficSource.adwordsClickInfo.slot", "trafficSource.campaign",
            "trafficSource.keyword", "trafficSource.medium", 
            "trafficSource.referralPath", "trafficSource.source",
            'trafficSource.adwordsClickInfo.isVideoAd', 'trafficSource.isTrueDirect']
for col in cat_cols:
    print(col)
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(train_df[col].values.astype('str')))
    train_df[col] = lbl.transform(list(train_df[col].values.astype('str')))
    


num_cols = ["totals.hits", "totals.pageviews", "visitNumber", "visitStartTime", 'totals.bounces',  'totals.newVisits']    
for col in num_cols:
    train_df[col] = train_df[col].astype(float)
   

channelGrouping
device.browser
device.deviceCategory
device.operatingSystem
device.isMobile
geoNetwork.city
geoNetwork.continent
geoNetwork.country
geoNetwork.metro
geoNetwork.networkDomain
geoNetwork.region
geoNetwork.subContinent
trafficSource.adContent
trafficSource.adwordsClickInfo.adNetworkType
trafficSource.adwordsClickInfo.gclId
trafficSource.adwordsClickInfo.page
trafficSource.adwordsClickInfo.slot
trafficSource.campaign
trafficSource.keyword
trafficSource.medium
trafficSource.referralPath
trafficSource.source
trafficSource.adwordsClickInfo.isVideoAd
trafficSource.isTrueDirect


In [ ]:
train_df.to_csv('dataset.csv')

In [ ]:
Y = np.log1p(train_df["totals.transactionRevenue"].astype('float'))
X=train_df.drop(["totals.transactionRevenue"],axis=1)##Eliminamos la salida de los datos de las caracteristicas
#X=X.drop(["date"],axis=1)

In [ ]:
groupsTrain = X['fullVisitorId']
group_kfold = GroupKFold(n_splits=2)

In [ ]:
train_index, test_index = next(GroupKFold(n_splits=4).split(X,Y, groupsTrain)) 
X_train = X.iloc[train_index]
Y_train= Y.iloc[train_index]
X_test = X.iloc[test_index]
Y_test = Y.iloc[test_index]   
    

In [ ]:
X_train.shape

(677739, 33)

In [ ]:
groups = X_train['fullVisitorId']
group_kfold= GroupKFold(n_splits=3).split(X_train, Y_train, groups)

In [ ]:
# Grid de hiperparámetros evaluados
# ==============================================================================
param_grid = {'max_depth'     : [3,6,9,12,15,18,21],
              'n_estimators' : [20,30,40,50,60,70,80,90,100]
             }

# Búsqueda por grid search con validación cruzada
# ==============================================================================
grid = GridSearchCV(
        estimator  = RandomForestRegressor(
                        random_state        = 0,
                       
                    ),
        param_grid = param_grid,
        scoring    = 'neg_root_mean_squared_error',
        n_jobs     =  40,
        cv         = group_kfold, 
        refit      = True,
        verbose    = 2,
        return_train_score = True
       )

In [ ]:
groups1=X_train
Model1=grid.fit(X_train,Y_train,groups1['fullVisitorId'])

Fitting 3 folds for each of 63 candidates, totalling 189 fits


[Parallel(n_jobs=40)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=40)]: Done  82 tasks      | elapsed:  9.9min
[Parallel(n_jobs=40)]: Done 189 out of 189 | elapsed: 24.8min finished


In [ ]:
TrainValRMSE=grid.cv_results_

In [ ]:
Model1.best_params_

{'max_depth': 9, 'n_estimators': 100}

In [ ]:
ErrorValRMSE=-1*(TrainValRMSE.get('split0_test_score')[17]+TrainValRMSE.get('split1_test_score')[17]+TrainValRMSE.get('split2_test_score')[17])/3
ErrorValRMSE

1.6674671943662265

In [ ]:
ErrorTrainRMSE=-1*(TrainValRMSE.get('split0_train_score')[17]+TrainValRMSE.get('split1_train_score')[17]+TrainValRMSE.get('split2_train_score')[17])/3
ErrorTrainRMSE

1.650508074573444

In [ ]:
-1*Model1.best_score_#confirma el calculo del error de validacion

1.6445736878931267

In [ ]:
Model=Model1.best_estimator_

In [ ]:
pred_test_y1= Model.predict(X_test)

In [ ]:
pred_test_y1[pred_test_y1<0]=0
print(f"RMSE= {np.sqrt(metrics.mean_squared_error(Y_test,pred_test_y1))}")
print(f"MAE= {metrics.mean_absolute_error(Y_test,pred_test_y1)}")

RMSE= 1.6789441683502193
MAE= 0.31582001779550084


In [ ]:
test_pred_df = pd.DataFrame({"fullVisitorId":X_test["fullVisitorId"].values})
test_pred_df["transactionRevenue"] = (Y_test.values)
test_pred_df["PredictedRevenue"] = (pred_test_y1)
test_pred_df = test_pred_df.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()
print('RMSE')
print(np.sqrt(metrics.mean_squared_error(test_pred_df["transactionRevenue"].values, test_pred_df["PredictedRevenue"].values)))
print('MAE')
print((metrics.mean_absolute_error((test_pred_df["transactionRevenue"].values ),(test_pred_df["PredictedRevenue"].values))))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



RMSE
2.248395887305349
MAE
0.3325213441820915


Evaluacion MAE

In [ ]:
groups3 = X_train['fullVisitorId']
group_kfold1= GroupKFold(n_splits=3).split(X_train, Y_train, groups3)

In [ ]:
# Grid de hiperparámetros evaluados
# ==============================================================================
param_grid = {'max_depth'     : [3,6,9,12,15,18,21],
              'n_estimators' : [20,30,40,50,60,70,80,90,100]
             }

# Búsqueda por grid search con validación cruzada
# ==============================================================================
grid1 = GridSearchCV(
        estimator  = RandomForestRegressor(
                        random_state        = 0,
                       
                    ),
        param_grid = param_grid,
        scoring    = 'neg_mean_absolute_error',###Pediente de realizar la funcion de error make,'mean_absolute_error'
        n_jobs     =  40,
        cv         = group_kfold1, 
        refit      = True,
        verbose    = 2,
        return_train_score = True
       )

In [ ]:
groups2=X_train
Model2=grid1.fit(X_train,Y_train, groups2['fullVisitorId'])

Fitting 3 folds for each of 63 candidates, totalling 189 fits


[Parallel(n_jobs=40)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=40)]: Done  82 tasks      | elapsed:  9.9min
[Parallel(n_jobs=40)]: Done 189 out of 189 | elapsed: 24.7min finished


In [ ]:
TrainValMAE=grid1.cv_results_

In [ ]:
Model2.best_params_

{'max_depth': 12, 'n_estimators': 80}

In [ ]:
ErrorValMAE=-1*(TrainValMAE.get('split0_test_score')[33]+TrainValMAE.get('split1_test_score')[33]+TrainValMAE.get('split2_test_score')[33])/3
ErrorValMAE

0.3041400556703015

In [ ]:
ErrorTrainMAE=-1*(TrainValMAE.get('split0_train_score')[33]+TrainValMAE.get('split1_train_score')[33]+TrainValMAE.get('split2_train_score')[33])/3
ErrorTrainMAE

0.24001275186814544

In [ ]:
-1*Model2.best_score_#confirma el calculo del error de validacion

0.3041400556703015

In [ ]:
ModelMAE=Model2.best_estimator_

In [ ]:
pred_test_y2= ModelMAE.predict(X_test)

In [ ]:
pred_test_y2[pred_test_y2<0]=0
print(f"RMSE= {np.sqrt(metrics.mean_squared_error(Y_test,pred_test_y2))}")
print(f"MAE= {metrics.mean_absolute_error(Y_test,pred_test_y2)}")

RMSE= 1.6736908987191295
MAE= 0.3110373034069749


In [ ]:
test_pred_df = pd.DataFrame({"fullVisitorId":X_test["fullVisitorId"].values})
test_pred_df["transactionRevenue"] =  (Y_test.values)
test_pred_df["PredictedRevenue"] = (pred_test_y2)
test_pred_df = test_pred_df.groupby("fullVisitorId")["transactionRevenue", "PredictedRevenue"].sum().reset_index()
print('RMSE')
print(np.sqrt(metrics.mean_squared_error(test_pred_df["transactionRevenue"].values, test_pred_df["PredictedRevenue"].values)))
print('MAE')
print((metrics.mean_absolute_error((test_pred_df["transactionRevenue"].values ),(test_pred_df["PredictedRevenue"].values))))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.



RMSE
2.223927574136908
MAE
0.32847641221568924
